<a href="https://colab.research.google.com/github/eduion/AIOT/blob/main/HW4/HW4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret
!pip install catboost
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.metrics import accuracy_score

def simplify_ages(df):
    df['Age'] = df['Age'].fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young_Adult', 'Adult', 'Senior']
    categories = pd.cut(df['Age'], bins, labels=group_names)
    df['Age'] = categories
    return df

def simplify_cabins(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0])
    return df

def simplify_fares(df):
    df['Fare'] = df['Fare'].fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df['Fare'], bins, labels=group_names)
    df['Fare'] = categories
    return df

def format_name(df):
    df['Lname'] = df['Name'].apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df['Name'].apply(lambda x: x.split(' ')[1])
    return df

def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = format_name(df)
    df = drop_features(df)
    return df

def rename_features(df):
    df.columns = df.columns.str.replace(r'[^\w]', '_', regex=True)
    return df

# 示例：讀取並處理數據
if __name__ == "__main__":
    # 載入數據集
    data_train = pd.read_csv('./train.csv')  # 將路徑改為實際數據的位置
    data_test = pd.read_csv('./test.csv')

    # 轉換特徵
    data_train = transform_features(data_train)
    data_train = rename_features(data_train)

    data_test = transform_features(data_test)
    data_test = rename_features(data_test)

    # 檢查列名
    print(data_train.columns)

    train_data, test_data = train_test_split(data_train.drop(['PassengerId'], axis=1), random_state=100, train_size=0.8)

    # 查看前幾行數據
    print(train_data.head())

    clf1 = setup(data=train_data,
                 target='Survived',
                 categorical_features=['Pclass', 'Sex', 'Age', 'Fare', 'Cabin', 'Lname', 'NamePrefix'])
    compare_models(fold=5)

    # 創建模型
    lr = create_model('lr', fold=5)
    knn = create_model('knn', fold=5)
    nb = create_model('nb', fold=5)
    dt = create_model('dt', fold=5)
    svm = create_model('svm', fold=5)
    rbfsvm = create_model('rbfsvm', fold=5)
    gpc = create_model('gpc', fold=5)
    mlp = create_model('mlp', fold=5)
    ridge = create_model('ridge', fold=5)
    rf = create_model('rf', fold=5)
    qda = create_model('qda', fold=5)
    ada = create_model('ada', fold=5)
    lda = create_model('lda', fold=5)
    gbc = create_model('gbc', fold=5)
    et = create_model('et', fold=5)
    xgboost = create_model('xgboost', fold=5)
    catboost = create_model('catboost', fold=5)

    # 第二層用 xgboost
    stacker_all = stack_models(
        estimator_list=[lr, knn, nb, dt, svm, rbfsvm, gpc, mlp, ridge, rf, qda, ada, lda, gbc, et, xgboost, catboost],
        meta_model=xgboost)

    pred = predict_model(stacker_all, data=test_data)
    accuracy_score(pred['Survived'], pred['prediction_label'])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.1 MB/s eta 0:00:00
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Cabin', 'Lname', 'NamePrefix'],
      dtype='object')
     Survived  Pclass     Sex          Age  SibSp  Parch        Fare Cabin  \
408         0       3    male      Student      0      0  1_quartile     N   
480         0       3    male        Child      5      2  4_quartile     N   
510         1       3    male  Young_Adult      0      0  1_quartile     N   
609         1       1  female        Adult      0      0  4_quartile     C   
547         1       2    male      Unknown      0      0  2_quartile     N   

          Lname NamePrefix  
408  Birkeland,        Mr.  
480    Goodwin,    Master.  
510       Daly,        Mr.  
609     Shutes,      Miss.  
547       Padro          y  


,Description,Value
0,Session id,5456
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(712, 10)"
4,Transformed data shape,"(712, 54)"
5,Transformed train set shape,"(498, 54)"
6,Transformed test set shape,"(214, 54)"
7,Numeric features,2
8,Categorical features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8232,0.8781,0.7213,0.7887,0.7515,0.6151,0.6182,1.2740
ridge,Ridge Classifier,0.8152,0.8700,0.6789,0.8008,0.7337,0.5938,0.5995,0.1780
knn,K Neighbors Classifier,0.8012,0.8497,0.6363,0.7930,0.7055,0.5584,0.5664,0.3080
et,Extra Trees Classifier,0.7732,0.8132,0.6366,0.7300,0.6788,0.5048,0.5088,0.5740
rf,Random Forest Classifier,0.7611,0.8206,0.5993,0.7209,0.6504,0.4723,0.4797,0.3800
catboost,CatBoost Classifier,0.7510,0.7768,0.5780,0.7073,0.6346,0.4492,0.4555,1.2880
gbc,Gradient Boosting Classifier,0.7430,0.7351,0.6100,0.6741,0.6401,0.4412,0.4427,0.3000
xgboost,Extreme Gradient Boosting,0.7330,0.7350,0.5728,0.6667,0.6145,0.4130,0.4165,0.2980
ada,Ada Boost Classifier,0.7229,0.7465,0.5400,0.6657,0.5924,0.3867,0.3939,0.2880
svm,SVM - Linear Kernel,0.7189,0.7448,0.4976,0.7450,0.5379,0.3617,0.4020,0.1780


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8400,0.8782,0.7368,0.8235,0.7778,0.6534,0.6558
1,0.8500,0.9003,0.8158,0.7949,0.8052,0.6833,0.6834
2,0.8200,0.8719,0.7568,0.7568,0.7568,0.6139,0.6139
3,0.7677,0.8446,0.5676,0.7500,0.6462,0.4781,0.4884
4,0.8384,0.8956,0.7297,0.8182,0.7714,0.6471,0.6496
Mean,0.8232,0.8781,0.7213,0.7887,0.7515,0.6151,0.6182
Std,0.0294,0.0198,0.0826,0.0305,0.0549,0.0720,0.0686


...

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.7716,0.5789,0.7097,0.6377,0.4498,0.4553
1,0.7300,0.7428,0.5000,0.7037,0.5846,0.3930,0.4056
2,0.7600,0.7909,0.6486,0.6857,0.6667,0.4794,0.4798
3,0.7677,0.8019,0.5676,0.7500,0.6462,0.4781,0.4884
4,0.7475,0.7770,0.5946,0.6875,0.6377,0.4454,0.4482
Mean,0.7510,0.7768,0.5780,0.7073,0.6346,0.4492,0.4555
Std,0.0128,0.0201,0.0479,0.0232,0.0271,0.0314,0.0291


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8200,0.8158,0.6842,0.8125,0.7429,0.6060,0.6113
1,0.7600,0.7649,0.6316,0.7059,0.6667,0.4801,0.4819
2,0.8200,0.8829,0.7368,0.7778,0.7568,0.6141,0.6146
3,0.7000,0.8149,0.3684,0.7000,0.4828,0.2991,0.3296
4,0.7400,0.7538,0.7368,0.6364,0.6829,0.4646,0.4682
5,0.7800,0.8565,0.5263,0.8333,0.6452,0.4973,0.5248
6,0.7200,0.7165,0.5263,0.6667,0.5882,0.3805,0.3866
7,0.7400,0.6806,0.3889,0.7778,0.5185,0.3665,0.4078
8,0.7755,0.8181,0.6667,0.7059,0.6857,0.5113,0.5118


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Stacking Classifier,0.8268,0.8124,0.7333,0.8333,0.7801,0.6382,0.6418
